01 Working File

In [2]:
import numpy as np
import plotly.graph_objects as go

from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import rsi_tv

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [4]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="30m",
    candles_to_dl=300,
)

datetimes = candles[:,CandleBodyType.Timestamp].astype('datetime64[ms]')
close_prices = candles[:,CandleBodyType.Close]

In [3]:
rsi_is_below = 50
rsi_length = 14

In [6]:
rsi = rsi_tv(
    source=close_prices,
    length=rsi_length
)

rsi = np.around(rsi,1)
rsi

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, 42.20, 42.30, 41.40, 43.20, 36.80, 39.00, 36.60, 35.70, 33.80,
       35.70, 46.40, 44.20, 48.10, 47.60, 49.10, 55.10, 54.40, 53.10,
       51.10, 52.10, 48.90, 48.90, 48.40, 49.70, 48.90, 46.80, 52.10,
       49.10, 49.10, 52.50, 52.90, 57.50, 57.80, 63.10, 58.00, 60.00,
       53.80, 56.20, 56.60, 68.60, 65.30, 67.10, 63.10, 59.50, 51.90,
       55.40, 53.20, 47.70, 48.60, 47.20, 45.50, 44.20, 44.30, 45.80,
       38.80, 43.90, 41.80, 36.20, 40.30, 31.60, 35.60, 28.60, 38.40,
       35.40, 40.60, 40.00, 39.80, 35.70, 38.50, 42.70, 39.70, 36.90,
       35.40, 39.90, 39.40, 46.20, 45.50, 44.00, 42.90, 45.40, 43.20,
       45.60, 40.70, 39.40, 34.60, 28.40, 31.10, 43.20, 48.40, 59.60,
       67.30, 55.20, 55.30, 57.80, 57.10, 54.70, 57.60, 54.60, 46.80,
       46.50, 39.50, 38.10, 33.60, 28.80, 29.80, 29.20, 26.80, 21.40,
       22.10, 20.60, 29.10, 31.10, 36.10, 36.20, 35.90, 31.20, 42.00,
       41.50,

In [7]:
prev_rsi = np.roll(rsi,1)
prev_rsi[0] = np.nan

prev_prev_rsi = np.roll(prev_rsi,1)
prev_prev_rsi[0] = np.nan

In [11]:
falling = prev_prev_rsi > prev_rsi
rising = rsi > prev_rsi
is_below = rsi < rsi_is_below

In [16]:
entries = np.where(is_below & falling & rising, True, False)

In [18]:
entry_signals = np.where(entries, rsi, np.nan)

In [19]:
exit_prices = np.full_like(rsi, np.nan)

In [20]:
fig = go.Figure()

In [21]:
fig.add_scatter(
    x=datetimes,
    y=rsi,
    name="rsi",
    line_color="yellow",
)

In [23]:
fig.add_scatter(
    x=datetimes,
    y=entry_signals,
    mode="markers",
    name="entries",
    marker=dict(
        size=12,
        symbol="circle",
        color="#00F6FF",
        line=dict(
            width=1,
            color="DarkSlateGrey",
        ),
    ),
)

In [24]:
fig.add_hline(
    y=rsi_is_below,
    opacity=0.3,
    line_color="red",
)

In [25]:
fig.update_layout(
    height=500,
    xaxis_rangeslider_visible=False,
    title=dict(
        x=0.5,
        text="Long Signals",
        xanchor="center",
        font=dict(
            size=50,
        ),
    ),
)